In [1]:
from lmnav.models.modeling_llama import LlamaForCausalLM
import torch
from torch.nn import functional as F
from transformers import LlamaTokenizer
from lmnav.common.utils import catchtime
from lmnav.models.modeling_clip import CLIPVisionModel 
# from transformers import CLIPVisionModel
from lmnav.common.utils import convert_weights_to_fp16

/coc/testnvme/pputta7/mambaforge/envs/lmnav/lib/python3.9/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


[2023-12-22 14:08:25,416] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model = LlamaForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype=torch.bfloat16).cuda()
tokenizer = LlamaTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype=torch.bfloat16)
model.eval()

for p in model.parameters():
    p.requires_grad = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
input_ids = tokenizer("hello what", return_tensors='pt').input_ids.repeat(10, 1).cuda()
input_embds = model.get_input_embeddings()(input_ids)
input_embds.shape

torch.Size([10, 3, 4096])

In [11]:
with catchtime("test"):
    data = model(inputs_embeds=input_embds)

TimeIt [test]: 0.034 seconds


In [7]:
backbone = CLIPVisionModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.bfloat16).cuda()
# convert_weights_to_fp16(backbone)
backbone.eval()

for p in backbone.parameters():
    p.requires_grad = False

In [11]:
envs = 10
imgs = torch.rand(envs, 3, 224, 224).cuda()
with catchtime("image encodings"):
    out = backbone(pixel_values=imgs)

torch.save(out, 'tmp_hf.pt')
# torch.save(out, 'tmp_fa.pt')

TimeIt [image encodings]: 0.027 seconds


In [13]:
x = torch.load('tmp_hf.pt')
y = torch.load('tmp_fa.pt')

In [16]:
torch.allclose(x.pooler_output, y.pooler_output, atol=1e-1, rtol=1e-1)

False

In [19]:
torch.sqrt(torch.var(torch.abs(x.pooler_output - y.pooler_output)))

tensor(0.0986, device='cuda:0', dtype=torch.bfloat16)

In [20]:
x.pooler_output - y.pooler_output

tensor([[ 0.0247, -0.0547,  0.1406,  ..., -0.1797,  0.0391, -0.0889],
        [ 0.1006,  0.0156,  0.0703,  ..., -0.0547, -0.0273, -0.0552],
        [ 0.1787, -0.3125,  0.2578,  ...,  0.0352, -0.2070,  0.0391],
        ...,
        [ 0.0840, -0.1172,  0.1992,  ...,  0.0781, -0.2070,  0.0508],
        [ 0.0630, -0.1875,  0.3184,  ..., -0.0234, -0.0078, -0.1104],
        [ 0.0186, -0.0312,  0.0156,  ...,  0.0156,  0.0430, -0.0845]],
       device='cuda:0', dtype=torch.bfloat16)

In [21]:
x.pooler_output

tensor([[-0.0530,  1.2734,  0.6875,  ..., -0.9062, -0.8828,  0.1631],
        [ 0.0757,  1.1094,  0.7969,  ..., -0.8711, -0.9883,  0.0845],
        [ 0.2715,  0.7969,  0.9141,  ..., -0.5430, -1.0938,  0.2539],
        ...,
        [ 0.0537,  1.0625,  0.8398,  ..., -0.7031, -1.0078,  0.3574],
        [ 0.0718,  1.0078,  0.7852,  ..., -0.7930, -0.8359,  0.0593],
        [ 0.1309,  1.0625,  0.6914,  ..., -0.7617, -0.9023,  0.0796]],
       device='cuda:0', dtype=torch.bfloat16)